# Zone<越战越勇>

* 使用 C++ 重构
* 语音模型从头训练
* 视觉模型使用YOLOv5

## 0. 初始化
### 0.A 拉取最新代码

In [55]:
%cd ~/traffic-monitor
!git pull

/home/nvidia/traffic-monitor
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/BestOwl/traffic-monitor
   36133d4..f134fda  master     -> origin/master
Updating 36133d4..f134fda
Fast-forward
 src/cpp/TrafficDetect.cpp | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)


### 0.B 定义模型路径

In [113]:
cv_engine_path = '~/yolov5s-dgx4-best@nano-sky.engine'

## 1. 编译
### 1.A 生成CMake缓存

In [45]:
!rm -r ~/traffic-monitor/build
!mkdir ~/traffic-monitor/build
%cd ~/traffic-monitor/build
!cmake -DCMAKE_BUILD_TYPE=RELEASE ..

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/home/nvidia/traffic-monitor/build
-- The C compiler identification is GNU 8.4.0
-- The CXX compiler identification is GNU 8.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.1.1") 
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pth

### 1.B 编译 C++ 代码

In [126]:
%cd ~/traffic-monitor/build
!make

/home/nvidia/traffic-monitor/build
[ 14%] Building NVCC (Device) object src/cpp/CMakeFiles/yoloplugin.dir/__/__/tensorrtx/yolov5/yoloplugin_generated_yololayer.cu.o
/home/nvidia/traffic-monitor/src/cpp/../../tensorrtx/yolov5/yololayer.h(85): warning: function "nvinfer1::IPluginV2Ext::configurePlugin(const nvinfer1::Dims *, int, const nvinfer1::Dims *, int, const nvinfer1::DataType *, const nvinfer1::DataType *, const __nv_bool *, const __nv_bool *, nvinfer1::PluginFormat, int)" is hidden by "nvinfer1::YoloLayerPlugin::configurePlugin" -- virtual function override intended?

/home/nvidia/traffic-monitor/src/cpp/../../tensorrtx/yolov5/yololayer.cu(59): warning: variable "a" was declared but never referenced

/home/nvidia/traffic-monitor/src/cpp/../../tensorrtx/yolov5/yololayer.cu(84): warning: variable "a" was declared but never referenced

/home/nvidia/traffic-monitor/src/cpp/../../tensorrtx/yolov5/yololayer.h(85): warning: function "nvinfer1::IPluginV2Ext::configurePlugin(const nvinfer

### 1.C 测试运行

In [21]:
%cd ~/traffic-monitor/build
!src/cpp/TrafficDetect

/home/nvidia/traffic-monitor/build
Bad arguments!
#zone<越战越勇> TrafficDetect 2.0
Usage: TrafficDetect <task_flag> <input_file> <engine_file>
    engine_file: path to model TensorRT engine file
    task_flag: 0 indicates picture, 1 indicates video
    input_file: path to input file
    output_label_path: path to output



## 2. 检测图片并输出label文件

* YOLOv5

### 定义图片文件夹路径

In [127]:
img_dir = '~/0_NVDC_Visual/val/images'
out_label_dir = '~/0_NVDC_Visual/mAP/input/detection-results'

### 开始运行

In [128]:
%cd ~/traffic-monitor/build
!rm -r {out_label_dir}/*
!src/cpp/TrafficDetect {cv_engine_path} 2 {img_dir} {out_label_dir}

/home/nvidia/traffic-monitor/build
Loading engine file...
Deserializing engine...
Deserialize required 4867887 microseconds.
Successfully loaded engine.
         Engine Summary         
bindings: 2
maxBatchSize: 1
input[0]: 3x384x384    1769472 bytes
output[1]: 6001x1x1    24004 bytes
Start detection!
Completed!


In [83]:
!rm -r {out_label_dir}/*

rm: cannot remove '/home/nvidia/0_NVDC_Visual/mAP/input/detection-results/*': No such file or directory


### 计算 mAP

In [129]:
!python3 ~/0_NVDC_Visual/mAP/main.py

bicycle AP  = 8.82%
pedestrian AP  = 53.62%
road_sign AP  = 68.70%
vehicle AP  = 85.10%
mAP = 54.06%
Figure(640x480)


## 3. 加载语音模型并识别语音文件
### 模型：从头训练 

### 定义模型和输入路径

In [63]:
asr_model_path = '~/asr_model2.nemo'
asr_input_path = '/home/nvidia/1_NVDC_ASR/testdata/sbqc_yipeng.wav'

### 开始运行

In [64]:
%cd '~'
asr_result = !python3 run_nemo.py --model {asr_model_path} --input {asr_input_path}
asr_result = (asr_result[-1].split('\'')[1])
print(asr_result)

target_cls_id = -1
if asr_result == "识别两轮车":
    target_cls_id = 0
elif asr_result == "识别汽车":
    target_cls_id = 1
elif asr_result == "识别行人":
    target_cls_id = 2
elif asr_result == "识别路牌":
    target_cls_id = 3

/home/nvidia
识别汽车


## 4. 根据识别到的语音指令检测视频（FPS测试）
### FPS 优化方法
* 使用异步计算（异步CUDA API）提高FPS
* CPU 和 GPU 同时计算减少阻塞时间
![异步推理FPS模型时序图.png](异步推理FPS模型时序图.png)

### 定义视频路径

In [116]:
cv_video_path = '~/0_NVDC_Visual/video2.mp4'

### 开始运行（同时计算FPS）

In [123]:
%cd ~/traffic-monitor/build
if target_cls_id == -1:
    !src/cpp/TrafficDetect {cv_engine_path} 1 {cv_video_path}
else:
    !src/cpp/TrafficDetect {cv_engine_path} 1 {cv_video_path} {target_cls_id}

/home/nvidia/traffic-monitor/build
Loading engine file...
Deserializing engine...
Deserialize required 4831358 microseconds.
Successfully loaded engine.
         Engine Summary         
bindings: 2
maxBatchSize: 1
input[0]: 3x384x384    1769472 bytes
output[1]: 6001x1x1    24004 bytes
Loading video frames to memory...
All frames have been read!
Read time: 0.529 seconds
Start detection!
  fps: 20.9958  

Finish!
Time elapsed: 6.096 seconds
Writing inference result to video files
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 13 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Write time: 4.145 seconds


In [103]:
!echo {target_cls_id}

1


### 将生成的视频转码，以便能够在Jupyter Notebook中查看
* 这里采用的是利用GPU加速的转码技术，将输出的视频转换到MP4格式，比单纯使用CPU进行转码的速度有大幅度提升

In [124]:
%cd ~/traffic-monitor/build
!rm result-ffmpeg4.mp4
!ffmpeg -i result.mp4 -vcodec libx264 -f mp4 result-ffmpeg4.mp4

/home/nvidia/traffic-monitor/build
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu/Linaro 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --

### 查看视频推理结果

In [125]:
%cd ~/traffic-monitor/build
from IPython.display import Video

Video("result-ffmpeg4.mp4", embed=True)

/home/nvidia/traffic-monitor/build
